In [2]:
import os
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import DirectoryLoader
# from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from yandex_chain import YandexEmbeddings
from langchain_community.vectorstores import FAISS

In [3]:
loader = DirectoryLoader("/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data",
                    glob="**/*.txt",
                    loader_cls=TextLoader,
                    show_progress=True,
                    loader_kwargs={"encoding":"utf-8"})

data = loader.load()

100%|██████████| 1834/1834 [00:00<00:00, 20210.33it/s]


In [4]:
data[0]

Document(page_content=' \n  Автор  ООО "НПЦ "1С"  Концепция  миграции данных Инструкция \nпользователей «D -1C1-1.10.39 Сверка \nвзаиморасчетов с контрагентами»  Страница  14  \nСтраниц  52  \nВерсия   5  \n  \n Инструкция пользователей D -1C1 -1.10.39  \nСверка взаиморасчетов с контрагентами  \n  \n  \n   \n \nРисунок 5: Вкладка «Основное» документа «Акт инвентаризации расчетов»   \nНа вкладке «Счета » устанавливаются счета расчета, по которым необходимо \nпровести инвентаризацию. По кнопке « Настроить » можно выполнить настройку счетов \n(Рисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов» ).  \n \nРисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов»   \nПри нажатии на кнопку « Перезаполнить » происходит заполнение табличных частей \nна вкладке « Счета » и « Задолженность » (Рисунок 7: Заполненная вкладка «Счета» ).  \n', metadata={'source': '/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data/Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов_с_контрагент

In [9]:
text_splitter=RecursiveCharacterTextSplitter( # TODO: semantic chunking
    chunk_size=1000, # TODO: tune this
    chunk_overlap=70
)

chunks = text_splitter.split_documents(data)

In [10]:
chunks[0]

Document(page_content='Автор  ООО "НПЦ "1С"  Концепция  миграции данных Инструкция \nпользователей «D -1C1-1.10.39 Сверка \nвзаиморасчетов с контрагентами»  Страница  14  \nСтраниц  52  \nВерсия   5  \n  \n Инструкция пользователей D -1C1 -1.10.39  \nСверка взаиморасчетов с контрагентами  \n  \n  \n   \n \nРисунок 5: Вкладка «Основное» документа «Акт инвентаризации расчетов»   \nНа вкладке «Счета » устанавливаются счета расчета, по которым необходимо \nпровести инвентаризацию. По кнопке « Настроить » можно выполнить настройку счетов \n(Рисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов» ).  \n \nРисунок 6: Вкладка «Счета» документа «Акт инвентаризации расчетов»   \nПри нажатии на кнопку « Перезаполнить » происходит заполнение табличных частей \nна вкладке « Счета » и « Задолженность » (Рисунок 7: Заполненная вкладка «Счета» ).', metadata={'source': '/home/yagor/Рабочий стол/atomic/AtomHack/data/txt_data/Инструкция_D_1C1_1_10_39_Сверка_взаиморасчетов_с_контрагентами/page_

In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
db = FAISS.from_documents(chunks, embeddings)

/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [40]:
dir_name = 'unstructured_vdb'
all_path_dir = os.path.join("..", "data", dir_name)
if not os.path.exists(all_path_dir):
    os.makedirs(all_path_dir)

In [46]:
faiss_vdb = "index"

In [47]:
db.save_local(all_path_dir, faiss_vdb)

### Тестирование

In [48]:
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

In [44]:
ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


create_txt_from_pdf.ipynb  rag_for_unstructured.ipynb


In [50]:
# /home/yagor/Рабочий стол/atomic/AtomHack/data/unstructured_vdb
db2 = FAISS.load_local(
    os.path.join("..", "data", 'unstructured_vdb'),
    embeddings,
    allow_dangerous_deserialization = True
)

In [51]:
query = "Что является целью интеграционного сценария «Массовая выгрузка документов из ЕОСДО?"
docs = db2.similarity_search(query)

In [52]:
print(docs[0].page_content)
page = docs[0].metadata["source"].split("/")[-1].split("_")[-1].split(".")[0]
file_name = docs[0].metadata['source'].split('/')[-2] + ".pdf"
print(f"Страница: {page}", f"Исходный документ: {file_name}")

Автор АО «Гринатом» Инструкция пользователей «Интеграция 1С ERP 2.0: 
Цифровой Росатом с ЕОСДО » Страница 10 
Страниц 35 
Версия  6 
1.2 Интеграционный сценарий «Массовая выгрузка доку ментов из 
ЕОСДО» 
Схематично интеграционный сценарий изображён на Рис унке 1.2: 
Рисунок 1.2. Сценарий «Массовая выгрузка документов  из ЕОСДО» 
Формирование 
реестра и отправка 
реестра в ЕОСДО 
Получение и 
обработка реестра Выгрузка скан-
образов 
документов на 
файловый сервер Подтверждение 
завершения 
выгрузки 
M .O.3 M .I.3
 
Целью интеграционного сценария «Массовая выгрузка д окументов из 
ЕОСДО» является автоматизация предоставления реестр а скан-образов 
документов в контролирующие органы. 
Зависимые бизнес-процессы:  
 D-1C1-1.10.19 Формирование налоговой декларации по НП; 
 D-1C1-1.10.15 Формирование налоговой декларации по НДС; 
 D-1C1-1.10.35 Учет средств целевого финансирования;  
 D-1C1-1.10.40 Формирование бухгалтерской отчетности .
Страница: 10 Исходный документ: Инструкция Интегра